In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

In [ ]:
from sklearn.preprocessing import StandardScaler
mnist_data, mnist_target = mnist.data.values, mnist.target.values
X, y = StandardScaler().fit_transform(mnist_data), mnist_target.astype(np.int)
y_ = np.zeros((y.size, 10))
y_[np.arange(y.size), y] = 1

## Implementing all layers of CNN totally

### image to col & col to image function

In [ ]:
def im2col(input_data, FH, FW, stride=1, pad=0):
  """
  input_data : (N, C, H, W)
  fh, fw : int
  stride, pad : int
  """
  N, C, H, W = input_data.shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
  col = np.zeros((N, OH, OW, C, FH, FW))

  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      col[:, h, w, :, :, :] = img[:, :, ii:ii+FH, jj:jj+FW]
  col = col.reshape(N*OH*OW, -1)
  return col

def col2im(col, input_shape, FH, FW, stride=1, pad=0):
  N, C, H, W = input_shape
  OH = (H + 2 * pad - FH) // stride + 1
  OW = (W + 2 * pad - FW) // stride + 1
  col = col.reshape(N, OH, OW, C, FH, FW)
  img = np.zeros((N, C, H+2*pad, W+2*pad))
  for h in range(OH):
    ii = h * stride
    for w in range(OW):
      jj = w * stride
      img[:,:,ii:ii+FH, jj:jj+FW] = col[:, h, w, :, :, :]
  return img[:, :, pad:pad+H, pad:pad+W]

### Activation and Evaluating function

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, X):
    self.mask = (X <= 0)
    self.out = X.copy()
    self.out[self.mask] = 0
    return self.out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

def softmax(x):
  C = np.max(x)
  exps = np.exp(x - C)
  if x.ndim == 1:
    sum_exps = np.sum(exps)
    return exps / sum_exps
  sum_exps = np.sum(exps, axis=1)
  return (exps.T / sum_exps).T

def cross_entropy_error(y, t):
  if y.ndim == 1:
    y = y.reshape(1, y.size)
    t = t.reshape(1, t.size)
  batch_size = t.shape[0]
  return -np.sum( t * np.log(y + 1e-4) ) / batch_size

### Affine function

In [ ]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.original_x_shape = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.original_x_shape = x.shape
    self.x = x.reshape(x.shape[0], -1)
    return np.dot(self.x, self.W) + self.b
  
  def backward(self, dout):
    self.db = np.sum(dout, axis=0)
    self.dw = np.dot(self.x.T, dout)
    dx = np.dot(dout, self.W.T)
    return dx.reshape(self.original_x_shape)

### SoftmaxWithLoss function

In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    return dx

### Initor function

In [ ]:
def gaussianInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) * weight_init_std

def xavierInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) / np.sqrt(input_size)

def heInit(input_size, output_size, weight_init_std=0.01):
  return np.random.randn(input_size, output_size) * np.sqrt( 2 / input_size)

### ANN based TwoLayerNetwork

In [ ]:
from collections import OrderedDict

class TwoLayerNetwork:
  def __init__(self, layer_num, layer_size=None, initor="Gaussian", weight_init_std=0.01):
    self.layer_num = layer_num
    self.layer_size = layer_size
    self.params = {}
    init_functions = {'Gaussian':gaussianInit, 'Xavier':xavierInit, 'He':heInit}
    for i in range(1, self.layer_num+1):
      self.params['W'+str(i)] = init_functions[initor](self.layer_size[i-1], self.layer_size[i], weight_init_std)
      self.params['b'+str(i)] = np.zeros(self.layer_size[i])
    self.layers = OrderedDict()
    for i in range(1, self.layer_num+1):
      self.layers['Affine'+str(i)] = Affine(self.params['W'+str(i)], self.params['b'+str(i)])
      self.layers['Relu'+str(i)] = Relu()
    self.lastLayer = SoftmaxWithLoss()
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x
  
  def loss(self, x, t):
    x = self.predict(x)
    return self.lastLayer.forward(x, t)

  def accuracy(self, x, t):
    ret = self.predict(x)
    y = np.argmax(ret, axis=1)
    t = np.argmax(t, axis=1)
    return np.sum(y == t) / x.shape[0]

  def gradient(self, x, t):
    self.loss(x, t)

    dout = 1
    dout = self.lastLayer.backward(dout)
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    grads = {}
    for i in range(1, self.layer_num+1):
      grads['W'+str(i)] = self.layers['Affine'+str(i)].dw
      grads['b'+str(i)] = self.layers['Affine'+str(i)].db
    return grads

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_, test_size=.1, stratify=y_)

In [ ]:
layer_num = 2
layer_size = [784, 50, 10]
net = TwoLayerNetwork(layer_num, layer_size, initor="He")
iter_num = 1000
lr = 0.005

batch_size = 200
epoch_size = max((iter_num // batch_size), 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []
for i in range(iter_num):
  targets = np.random.choice(X_train.shape[0], batch_size)
  X_batch, t_batch = X_train[targets], y_train[targets]

  grads = net.gradient(X_batch, t_batch)
  for ii in range(1, net.layer_num+1):
    net.params['W'+str(ii)] -= lr * grads['W'+str(ii)]
    net.params['b'+str(ii)] -= lr * grads['b'+str(ii)]
    loss = net.loss(X_batch, t_batch)
    train_loss_list.append(loss)

  if i % epoch_size == 0:
    train_acc = net.accuracy(X_train, y_train)
    test_acc = net.accuracy(X_test, y_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc)

0.10931746031746031 0.10371428571428572
0.1357936507936508 0.13157142857142856
0.16295238095238096 0.15957142857142856
0.19341269841269842 0.18942857142857142
0.22011111111111112 0.21871428571428572
0.24761904761904763 0.24428571428571427
0.2746984126984127 0.2712857142857143
0.2986190476190476 0.29942857142857143
0.3248412698412698 0.32485714285714284
0.34844444444444445 0.3477142857142857
0.36652380952380953 0.3668571428571429
0.38476190476190475 0.38642857142857145
0.4022857142857143 0.4034285714285714
0.4186666666666667 0.41942857142857143
0.4330952380952381 0.43614285714285717
0.44926984126984126 0.45171428571428573
0.46506349206349207 0.4654285714285714
0.4805714285714286 0.4817142857142857
0.49557142857142855 0.49585714285714283
0.5089523809523809 0.5085714285714286
0.5231587301587302 0.5222857142857142
0.5360952380952381 0.5342857142857143
0.5478253968253968 0.5445714285714286
0.558968253968254 0.5557142857142857
0.5695714285714286 0.5674285714285714
0.5800634920634921 0.577142

### Convolution layer

In [ ]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    self.x = x
    N, C, H, W = self.x.shape
    FN, C, FH, FW = self.W.shape
    out_h = (H + 2*self.pad - FH) // self.stride + 1
    out_w = (W + 2*self.pad - FW) // self.stride + 1
    
    self.col = im2col(x, FH, FW, self.stride, self.pad)
    self.col_W = self.W.reshape(FN, -1).T
    out = np.dot(self.col, self.col_W) + self.b

    out = out.reshape(N, out_h, out_w, FN).transpose(0,3,1,2)
    return out
  
  def backward(self, dout):
    N, C, H, W = self.x.shape
    FN, C, FH, FW = self.W.shape
    out_h = (H + 2*self.pad - FH) // self.stride + 1
    out_w = (W + 2*self.pad - FW) // self.stride + 1
    # step1
    dout = dout.transpose(0,2,3,1).reshape(-1, FN)
    # step2 for db
    self.db = np.sum(dout, axis=0)
    # step3 for self.col * self.col_W
    dcol = np.dot(dout, self.col_W.T)
    dcol_W = np.dot(self.col.T, dout)
    # step4 for dx, dw
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
    self.dw = dcol_W.T.reshape(FN, C, FH, FW)
    return dx

### Pooling layer

In [ ]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
  
  def forward(self, x):
    N, C, H, W = x.shape
    out_h = (H - self.pool_h) // self.stride + 1
    out_w = (W - self.pool_w) // self.stride + 1
    col = im2col(x, self.pool_h, self.pool_w, stride=self.stride, pad=self.pad)
    col = col.reshape(-1, self.pool_h * self.pool_w)
    self.arg_max = np.argmax(col, axis=1)
    out = np.max(col, axis=1)
    out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
    self.x = x
    return out

  def backward(self, dout):
    N, C, H, W = self.x.shape
    out_h = (H - self.pool_h) // self.stride + 1
    out_w = (W - self.pool_w) // self.stride + 1
    dout = dout.transpose(0,2,3,1).reshape(N, out_h, out_w, C)

    dmax = np.zeros((dout.size, self.pool_h * self.pool_w))
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
    dmax = dmax.reshape(dout.shape + (self.pool_h * self.pool_w,))
    dcol = dmax.reshape(dmax.shape[0], dmax.shape[1], dmax.shape[2], -1)
    dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
    return dx

### Making CNN architecture

<Architecture>
input -- --> Conv -> ReLU -> Pooling -- --> Affine -> ReLU -- --> Affine -> Softmax

In [ ]:
from collections import OrderedDict

class SimpleConvNet:
  def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, \
               hidden_size=100, output_size=10, initor="Gaussian", weight_init_std=0.01):
    
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['pad']
    filter_stride = conv_param['stride']
    input_size = input_dim[1]
    conv_output_size = (input_size + 2 * filter_pad - filter_size) // filter_stride + 1
    pool_output_size = int(filter_num * (conv_output_size / 2)**2)  # pooling layer's size: 2, stride:2

    init_functions = {'Gaussin':gaussianInit, 'Xavier':xavierInit, 'He':heInit}
    self.params = {}
    # for Convolution layer
    self.params['W1'] = np.random.randn(filter_num, input_dim[0], filter_size, filter_size) * weight_init_std
    self.params['b1'] = np.zeros(filter_num)
    # for Affine layer 1
    self.params['W2'] = init_functions[initor](pool_output_size, hidden_size, weight_init_std)
    self.params['b2'] = np.zeros(hidden_size)
    # for Affine layer 2
    self.params['W3'] = init_functions[initor](hidden_size, output_size, weight_init_std)
    self.params['b3'] = np.zeros(output_size)

    self.layers = OrderedDict()
    self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], stride=filter_stride, pad=filter_pad)
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
    self.last_layer = SoftmaxWithLoss()
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def gradient(self, x, t):
    self.loss(x, t)

    dout = 1
    dout = self.last_layer.backward(dout)
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    grads = {}
    grads['W1'] = self.layers['Conv1'].dw
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dw
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dw
    grads['b3'] = self.layers['Affine2'].db

    return grads

In [ ]:
X_ = X.reshape(-1, 1, 28, 28)

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=.1, stratify=y_)

In [ ]:
net = SimpleConvNet(initor="He")
iter_num = 10000
lr = 0.01

batch_size = 100
epoch_size = max((iter_num // batch_size), 1)

train_loss_list = []
train_acc_list = []
test_acc_list = []
for i in range(iter_num):
  targets = np.random.choice(X_train.shape[0], batch_size)
  X_batch, t_batch = X_train[targets], y_train[targets]

  grads = net.gradient(X_batch, t_batch)
  for ii in range(1, 3+1):
    net.params['W'+str(ii)] -= lr * grads['W'+str(ii)]
    net.params['b'+str(ii)] -= lr * grads['b'+str(ii)]
  loss = net.loss(X_batch, t_batch)
  train_loss_list.append(loss)

  if i % epoch_size == 0:
    #train_acc = net.accuracy(X_traiㅑn, y_train)
    test_acc = net.accuracy(X_test, y_test)
    #train_acc_list.append(train_acc)
    #test_acc_list.append(test_acc)
    #print(train_acc, test_acc)
    print(test_acc)

0.10957142857142857
0.8172857142857143
0.8917142857142857
0.913
0.9248571428571428
0.9325714285714286
0.9361428571428572
0.9404285714285714
0.9447142857142857
0.948
0.9485714285714286
0.9532857142857143
0.9548571428571428
0.9555714285714285
0.9581428571428572
0.958
0.9598571428571429
0.9618571428571429
0.9625714285714285
0.9641428571428572
0.9642857142857143
0.9668571428571429
0.9671428571428572
0.9661428571428572
0.9705714285714285
0.968
0.9695714285714285
0.9695714285714285
0.969
0.9705714285714285
0.9732857142857143
0.9702857142857143
0.9718571428571429
0.9728571428571429
0.9721428571428572
0.9722857142857143
0.9734285714285714
0.9735714285714285
0.9732857142857143
0.9741428571428571
0.9747142857142858
0.9758571428571429
0.9745714285714285
0.9754285714285714
0.9758571428571429
0.9732857142857143
0.9757142857142858
0.9765714285714285
0.9765714285714285
0.9772857142857143
0.9778571428571429
0.9767142857142858
0.977
0.9778571428571429
0.9791428571428571
0.9777142857142858
0.98
0.978857